In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-06 01:49:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.61MB/s    in 0.2s    

2021-12-06 01:49:44 (5.61 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [10]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter("total_votes >= 20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [12]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
helpful_votes_df = filtered_vine_df.filter("helpful_votes/total_votes >= 0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [13]:
# Step 3: Create a new DF: condition => vine=='Y'
vine_Y_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [14]:
# Step 4: Create a new DF: condition => vine=='N'
vine_N_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [20]:
# Step 5.1: vine_reviews_Y: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews'
# Step 5.1.a: total number of reviews
vineY_total_reviews = vine_Y_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "vine_review_count")
vineY_total_reviews.show()

+-----------------+
|vine_review_count|
+-----------------+
|               94|
+-----------------+



In [23]:
# Step 5.1.b: 5-star reviews
vine_Y_5star = vine_Y_df.filter("star_rating == 5").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "5_star_vine_reviews")
vine_Y_5star.show()

+-------------------+
|5_star_vine_reviews|
+-------------------+
|                 48|
+-------------------+



In [33]:
vine_Y_5star_percent = (vine_Y_5star.collect()[0]["5_star_vine_reviews"] / vineY_total_reviews.collect()[0]["vine_review_count"]) * 100
round(vine_Y_5star_percent, 2)

51.06

In [34]:
# Step 5.2: vine_reviews_NO: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews '
# Step 5.2.a: total number of reviews
vineN_total_reviews = vine_N_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Non-Vine_review_count")
vineN_total_reviews.show()

+---------------------+
|Non-Vine_review_count|
+---------------------+
|                40471|
+---------------------+



In [35]:
# Step 5.2.a: number of 5-stars reviews
vine_N_5star = vine_N_df.filter("star_rating == 5").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "5_star_non-vine_reviews")
vine_N_5star.show()

+-----------------------+
|5_star_non-vine_reviews|
+-----------------------+
|                  15663|
+-----------------------+



In [37]:
vine_N_5star_percent = (vine_N_5star.collect()[0]["5_star_non-vine_reviews"] / vineN_total_reviews.collect()[0]["Non-Vine_review_count"]) * 100
round(vine_N_5star_percent, 2)

38.7

In [39]:
# Create data frame for % results
percentage_df = spark.createDataFrame([
                                       (51.1, 38.7)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                   51.1|                       38.7|
+-----------------------+---------------------------+



In [41]:
# Positivity Bias - additional calculations
# Vine Reviews

vine_Y_by_star = vine_Y_df.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "vine_reviews_by_star")
vine_Y_by_star.show()

+-----------+--------------------+
|star_rating|vine_reviews_by_star|
+-----------+--------------------+
|          1|                   1|
|          3|                  16|
|          5|                  48|
|          4|                  24|
|          2|                   5|
+-----------+--------------------+



In [42]:
# Vine Reviews (cont)
import pyspark.sql.functions as func
vine_Y_by_star_percentage = vine_Y_by_star.withColumn('% of total', func.round(vine_Y_by_star['vine_reviews_by_star']/ \
                                                (vineY_total_reviews.collect()[0]['vine_review_count']/100), 2)) \
                                                .orderBy(vine_Y_by_star['star_rating'].asc()).show()

+-----------+--------------------+----------+
|star_rating|vine_reviews_by_star|% of total|
+-----------+--------------------+----------+
|          1|                   1|      1.06|
|          2|                   5|      5.32|
|          3|                  16|     17.02|
|          4|                  24|     25.53|
|          5|                  48|     51.06|
+-----------+--------------------+----------+



In [44]:
# non-Vine Reviews
vine_N_by_star= vine_N_df.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "non-vine_reviews_by_star")
vine_N_by_star.show()

+-----------+------------------------+
|star_rating|non-vine_reviews_by_star|
+-----------+------------------------+
|          1|                   10303|
|          3|                    4379|
|          5|                   15663|
|          4|                    6738|
|          2|                    3388|
+-----------+------------------------+



In [47]:
# Non- Vine Reviews (cont)
import pyspark.sql.functions as func
vine_N_by_star_percentage = vine_N_by_star.withColumn('% of total', func.round(vine_N_by_star['non-vine_reviews_by_star']/ \
                                                (vineN_total_reviews.collect()[0]['Non-Vine_review_count']/100), 2)) \
                                                .orderBy(vine_N_by_star['star_rating'].asc()).show()

+-----------+------------------------+----------+
|star_rating|non-vine_reviews_by_star|% of total|
+-----------+------------------------+----------+
|          1|                   10303|     25.46|
|          2|                    3388|      8.37|
|          3|                    4379|     10.82|
|          4|                    6738|     16.65|
|          5|                   15663|      38.7|
+-----------+------------------------+----------+

